In [ ]:
import sys

# Add path for python to look into for modules installed using pip

sys.path.append('/usr/local/lib/python3.7/site-packages/')

import quandl
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import requests
import json
import datetime
import random

np.set_printoptions(threshold=sys.maxsize)

quandl.ApiConfig.api_key = "n2tNssPxEFC9-Ad79fo-" # keep this private

figsize = (15, 8)

pd.set_option('display.max_columns', None)

## Get ticker/RIC codes from different stock exchanges

In [ ]:
def Get_Codes(stock_exchange):
    
    data = pd.read_csv(str(stock_exchange)+".csv", encoding='unicode_escape')
    company_name = []

    for i in data["Symbol"]:
    
        company_name.append(i)

    codes = np.asarray(company_name)
    
    return codes


Get_Codes("NYSE")

Get_Codes("NASDAQ")

Get_Codes("ASE")

# We must add the codes at the end to make it compatible with Refinitiv data

def Add_Extension(codes, ext):
    
    newcodes = []
    
    for i in codes:
        
        newcodes.append(str(i) + str(ext))
        
    new = np.asarray(newcodes)
    
    return new   

Add_Extension(Get_Codes("NASDAQ"), ext=".O")

# We can now create a list of extensions to call the above function for each stock exchange:

# list can be found at: 
# http://training.thomsonreuters.com/portal/docs/pdf/raymondjames/Thomson_One_Exchange_List.pdf

# For the time being we have the 3 largest American Stock Exchanges:

# - NYSE: .N
# - NASDAQ: .O
# - ASE: .A

# Get Quandl data

NYSE_quandl = list(Get_Codes("NYSE"))
NASDAQ_quandl = list(Get_Codes("NASDAQ"))
ASE_quandl = list(Get_Codes("ASE"))

# Get Rifinitiv data

NYSE_rifinitiv = list(Add_Extension(Get_Codes("NYSE"), ext=".N"))
NASDAQ_rifinitiv = list(Add_Extension(Get_Codes("NASDAQ"), ext=".O"))
ASE_rifinitiv = list(Add_Extension(Get_Codes("ASE"), ext=".A"))

print('NYSE_quandl:', len(NYSE_quandl))
print('NASDAQ_quandl:', len(NASDAQ_quandl))
print('ASE_quandl:', len(ASE_quandl))

print('NYSE_rifinitiv:', len(NYSE_rifinitiv))
print('NASDAQ_rifinitiv:', len(NASDAQ_rifinitiv))
print('ASE_rifinitiv:', len(ASE_rifinitiv))

## Set time period

In [ ]:
# Start/end dates

start = '2010-10-15'
end = '2018-10-15'

oos_start = end
oos_end = '2019-10-15'

## Rifinitiv

In [ ]:
##### Rifinitiv #####

# The following values are populated for you by Data Science Accelerator. 
# They represent your demo-level access to the data.
# Please don't share this with anyone

# RESOURCE_ENDPOINT = 'https://dsa-stg-edp-api.fr-nonprod.aws.thomsonreuters.com/data/environmental-social-governance/v1/views/scores-full'
access_token = 'uGR7cxvvqJ4mgWwva5pPN184iGGigBhY8g4ThFu0' # personal key for Data Science Accelerator access to ESG

def get_data_request(url, requestData):
    '''HTTP GET request'''
    dResp = requests.get(url, headers = {'X-api-key': access_token}, params = requestData);       

    if dResp.status_code != 200:
        raise ValueError("Unable to get data. Code %s, Message: %s" % (dResp.status_code, dResp.text));
    else:
        print("Data access successful")
        jResp = json.loads(dResp.text);
        return jResp

def get_data(ric):
    '''Gets ESG scores for a specific RIC (company) code'''
    
    requestData = {
    "universe": ric
    };

    jResp = get_data_request(RESOURCE_ENDPOINT, requestData)

    data = jResp["data"]
    headers = jResp["headers"]    

    names = [headers[x]['title'] for x in range(len(headers))]

    df = pd.DataFrame(data, columns=names )
    
    return df

# Updating the API endpoint 

RESOURCE_ENDPOINT = 'https://dsa-stg-edp-api.fr-nonprod.aws.thomsonreuters.com/data/environmental-social-governance/v1/views/measures-full'

## Aggregate function

In [ ]:
# Read available Quandl ticker symbols

tickers = pd.read_csv('ticker_list.csv', sep=',')

In [ ]:
# Define essential loop function

def aggregate(asset1, asset2, start_date=start, end_date=end):
    
    # Quandl
    
    if 'EOD/'+asset1.replace('.', '_') in list(tickers['Quandl_Code']):
        
        try:
            quandl_data = quandl.get('EOD/'+asset1.replace('.', '_'), start_date=start_date, end_date=end_date)
        except:
            return 0
        
        # Get stocks with at least X Quandl entries
        
        if len(quandl_data) >= 500:
            quandl_data['log_ret'] = np.log(quandl_data.Adj_Close) - np.log(quandl_data.Adj_Close.shift(1))
            quandl_data['Date'] = pd.to_datetime(quandl_data.index, format='%Y-%m-%d')
            quandl_data['Year'] = quandl_data['Date'].dt.year
            quandl_yearly = quandl_data.groupby(['Year']).mean()
        else:
            return 0
    else:
        return 0
    
    # Refinitiv
    
    try:
        refinitiv_data = get_data(asset2)
    except:
        return 0
        
    if len(refinitiv_data) != 0:
        refinitiv_data['Date'] = pd.to_datetime(refinitiv_data['Period End Date'], format='%Y-%m-%d')
        refinitiv_data['Year'] = refinitiv_data['Date'].dt.year
        refinitiv_data['Period End Date'] = pd.to_datetime(refinitiv_data['Period End Date'], format='%Y-%m-%d')
        refinitiv_data = refinitiv_data[(refinitiv_data['Period End Date'] >= start) & 
                                        (refinitiv_data['Period End Date'] <= end)]
        refinitiv_data.set_index('Year', inplace=True)
        refinitiv_data.dropna(subset=['TRESG Combined Score'], inplace=True)
    else:
        return 0
        
    # Join datasets (if both available)
    
    all_data = quandl_yearly.join(refinitiv_data, on='Year', how='left')
    
    return all_data

## Quandl-Rifinitive loop

In [ ]:
n = 10

index = random.sample(population=range(0, len(NASDAQ_quandl)), k=n)

assets_quandl = [NASDAQ_quandl[i] for i in index]

assets_ref = [NASDAQ_rifinitiv[i] for i in index]

datasets = []

i = 1

for asset1, asset2 in zip(assets_quandl,assets_ref):
    
    print(i, '/', n, ':', asset1, ',', asset2)
    
    data_tmp = aggregate(asset1, asset2)
    
    i += 1
    
    if isinstance(data_tmp, pd.DataFrame):
        datasets.append(data_tmp)
        print(data_tmp.shape, '\n')
    else:
        continue

print('Done')

In [ ]:
if len(datasets) >= 1:
    
    # Get all column names

    col_names = []

    for d in datasets:
        col_names.append(d.columns)
    
    # Intersection of list of lists

    unique_colnames = list(set.intersection(*map(set, col_names)))
    
else:
    print('No datasets found')

In [ ]:
# Subset dataframes in datasets

new_data = []

for d in datasets:
    df_tmp = d[np.intersect1d(d.columns, unique_colnames)]
    df_tmp = df_tmp.loc[:, ~df_tmp.columns.duplicated()]
    new_data.append(df_tmp)
    
# Merge dataframes in new_data list

all_data = pd.concat(new_data, axis=0)

In [ ]:
all_data.shape

In [ ]:
all_data.head()

In [ ]:
# all_data.to_csv('aggr_data.csv', sep = ',')